# 背包问题(knapsack problem) -- 动态规划

参考链接:
1. [背包问题九讲](https://www.kancloud.cn/kancloud/pack)

## 第一讲 01背包问题
### 题目
有`N`件物品和一个容量为`V`的背包。第`i`件物品的费用是`c[i]`，价值是`w[i]`。求解将哪些物品装入背包可使价值总和最大。

### 基本思路
子问题定义状态：即`f[i][j]`表示前`i`件物品恰好放入一个容量为`V`的背包可以获得的最大价值。则其状态访问方程便是
```c
f[i][v] = max{f[i-1][v], f[i-1][v-c[i]]+w[i]}
```

只考虑第`i`件物品的策略(放或者不放)，那么就可以转化成一个只牵扯前`i-1`件物品的问题。
1. 如果不放第`i`件物品，那么问题就转化成了 -> “前`i-1`件物品放入容量为`V`的背包中”，价值为`f[i-1][v]`。
2. 如果放入第`i`件物品，那么问题就转化成了 -> "前`i-1`件物品放入剩下的容量为`v-c[i]`的背包中"，此时能获得的最大价值就是`f[i-1][v-c[i]]`再加上通过放入第`i`件物品获得的价值`w[i]`。

### 优化空间复杂度
- 时间复杂度：`O(VN)`，无法优化了
- 空间复杂度：`O(VN)`，还能优化到`O(V)`

分析以上思路，首先肯定得有一个主循环`i=1..N`，每次算出来二维数组`f[i][0..V]`的所有值。

在每次`i`循环时，只用一个`f[0..V]`数组保存结果，就能保证第`i`次循环后`f[v]`中表示的就是我们定义的`f[i][v]`状态。

- **问题：**由于`f[i][v]`是由前面的`f[i-1][v]`和`f[i-1][v-c[i]]`两个子问题递推而来的。如果是以`v=0..V`的顺序遍历，则不能保存`v`之前的`f[i-1][v-c[i]]`，因为已经被之前的值更新替换掉了。
- **解决方案：**逆序的推`f[v], v=V..0`就能保证`f[v]`时`f[v-c[i]`保存的状态是`f[i-1][v-c[i]]`的值。伪代码如下：

```c
for i=1..N
    for v=V..0
        f[v] = max{f[v], f[v-c[i]]+w[i]};
```

过程`ZeroOnePack`，表示处理一件01背包中的物品，两个参数`cost, weight`分别表示这件物品的费用和价值。
```c
func ZeroOnePack(cost, weight)
    for v=V..cost
        f[v] = max{f[v], f[v-cost]+weight}
```

之后的01背包伪代码可以写成：
```c
for i=1..N
    ZeroOnePack(c[i], w[i]);
```

### 初始化的细节问题
最优解的背包问题中，有两种不同的问法：
- "恰好装满背包"时的最优解
- 没有要求必须装满背包

区别：
- 恰好装满：初始化时，除了`f[0]`为`0`，其他的`f[1..V]`均为$-\infty$，这样能保证最终得到的`f[V]`是一种恰好装满背包的最优解。
- 没有要求：初始化时，`f[0..V]=0`。

如果要求背包恰好装满，那么此时只有容量为0的背包可能被价值为0的nothing“恰好装满”，其它容量的背包均没有合法的解，属于未定义的状态，它们的值就都应该是$-\infty了。

如果背包并非必须被装满，那么任何容量的背包都有一个合法解“什么都不装”，这个解的价值为0，所以初始时状态的值也就全部为0了。


### 一个常数优化
前面的伪代码中有 `for v=V..1`，可以将这个循环的下限进行改进。

当计算到第i个物品时，我们只需要知道`f[V-c[i]]`的值是多少，也就是说计算第`i-1`个物品的时候我们只需要计算出`f[V-c[i]]`的值就可以停止循环了。进一步，当处理第i个物品时只需要循环到：
$$MAX(V-\sum^{n}_{i+1}c_{i+1}, c_i)$$

```c
for i=1..N
    bound = max{V-sum{w[i..N]}, c[i]}
    for v = V..bound
```

[代码如下](https://www.jianshu.com/p/8d41d87fcbb7)：
```c
/** nyoj: 654*/
//c header
#include <cstdlib>
#include <cstdio>
#include <cstring>
//cpp header
#include <iostream>
#include <vector>
#include <algorithm>
#include <string>

using namespace std;
#define M 1100000 //Max bag's Capacity
#define N 120 //Max Item's amount
#define CLS(x,v) memset(x,v,sizeof(x))
typedef pair<int,int> ppi;

/**Cap is the bag's Capacity; SumCost is the sum of Item's cost*/
int dp[M],Cap,SumCost;
/** first is cost ,second is weight*/
int cmp(ppi x,ppi y)
{
    //return true will be Swap elements
    return x.first>y.first;
}
int ZeroOnePack(int cost,int weight)
{
    SumCost-=cost;
    int bound=max(Cap-SumCost,cost);
    for(int v=Cap;v>=bound;--v)
        dp[v]=max(dp[v],dp[v-cost]+weight);
}
int solve(vector<ppi> &Items)
{
    CLS(dp,0);
    for(int i=0;i<Items.size();i++)
        ZeroOnePack(Items[i].first,Items[i].second);
    //return Answer
    return dp[Cap];
}

int main()
{
    int T,n,cost,weight;
    vector<ppi>Items;
    //large input take effect obviously
    ios::sync_with_stdio(false);
    scanf("%d",&T);
    while(T--)
    {
        scanf("%d%d",&n,&Cap);
        SumCost=0;
        Items.clear();
        for(int i=0;i<n;i++)
        {
            scanf("%d%d",&cost,&weight);
            SumCost+=cost;
            Items.push_back(make_pair(cost,weight));
        }
        sort(Items.begin(),Items.end(),cmp);
        printf("Max experience: %d\n",solve(Items));
    }
    return 0;
}
```
---


## 第二讲 完全背包问题

---